In [ ]:
import timeit
import pickle
from tqdm.notebook import tqdm
import pandas as pd
import csv
import multiprocessing

# Creating Rank 1 points

In [ ]:
U1=elliptic_curves.rank(n=50000, rank=1, tors=0, labels=False)

In [ ]:
import multiprocessing
import csv

# Suppose U1 is your set of actual elliptic curve objects (not labels):
# U1 = [E1, E2, E3, ...]

def compute_generator_for_label(label):
    """
    Worker function that runs in a child process.
    Reconstructs the elliptic curve from the label and computes the first generator.
    """
    # Reconstruct the curve from the label
    E = EllipticCurve(label)
    return E.gens()[0]  # returns the generator P

def get_generator_with_timeout(label, timeout=12):
    """
    Spawns a new process to compute the generator via 'compute_generator_for_label'.
    If it doesn't finish within 'timeout' seconds, raises a TimeoutError.
    """
    with multiprocessing.Pool(processes=1) as pool:
        async_result = pool.apply_async(compute_generator_for_label, (label,))
        try:
            # This blocks up to 'timeout' seconds for the result
            return async_result.get(timeout=timeout)
        except multiprocessing.TimeoutError:
            raise TimeoutError(
                f"Rank computation for {label} timed out after {timeout} seconds."
            )


with open("Rank_1_points.csv", "w", newline="") as f:
    writer = csv.writer(f)
    # Write a header
    writer.writerow(["label", "point"])

    for E in tqdm(U1):
        label = E.label()
        try:
            # Attempt to compute the generator in a separate process
            P = get_generator_with_timeout(label, timeout=12)
            # If successful, record it
            writer.writerow([label, (P[0], P[1])])

        except TimeoutError as e:
            print(f"Skipping {label} — timed out.")
        except Exception as e:
            print(f"Skipping {label} — error: {e}")



# Lambda=1 test table

In [ ]:

def plus_test(E, p, P=None):
    if P is None:
        P = E.gens()[0]
    tam = E.tamagawa_product()
    N_p = E.Np(p)
    Q = tam * N_p * P
    a = Q[0].numerator() % (p**2)
    return (a**(p - 1) - 1 == 0)

def minus_test(E, p, P=None):
    if P is None:
        P = E.gens()[0]
    N_p = E.Np(p)
    Q = N_p * P
    d = sqrt(Q[0].denominator()) % (p**2)
    return (d == 0)

def do_all(label, p, point_str):
    """
    Child-process function that:
    1) Builds E from label
    2) Checks supersingularity
    3) Checks tamagawa_product mod p
    4) Parses the point (x, y)
    5) Returns (label, plus_test(E, p, P), minus_test(E, p, P))
    or None if skipped
    """
    from sage.all import EllipticCurve, QQ, sqrt
    E = EllipticCurve(label)
    
    if not E.is_supersingular(p):
        return None
    
    if E.tamagawa_product() % p == 0:
        return None
    
    point_str = point_str.strip("()")
    sx, sy = point_str.split(",")
    x = QQ(sx.strip())
    y = QQ(sy.strip())
    P = E(x, y)
    
    return (label, plus_test(E, p, P), minus_test(E, p, P))

def get_result_with_timeout(label, p, point_str, timeout=15):
    """
    Runs 'do_all(label, p, point_str)' in a separate process,
    enforcing a hard timeout of 'timeout' seconds.
    Returns either the tuple (label, plus_val, minus_val), None, or "TIMEOUT".
    """
    with multiprocessing.Pool(processes=1) as pool:
        async_result = pool.apply_async(do_all, (label, p, point_str))
        try:
            return async_result.get(timeout=timeout)
        except multiprocessing.TimeoutError:
            return "TIMEOUT"


In [ ]:
A=prime_range(40,70)
df = pd.read_csv("Rank_1_points.csv")
for p in A:
    results = []
    for _, row in tqdm(df.iterrows()):
        label = row["label"]
        point_str = row["point"]
        
        outcome = get_result_with_timeout(label, p, point_str, timeout=40)
        if outcome == "TIMEOUT" or outcome is None:
            # Skip if timed out or do_all(...) returned None
            continue
        
        lbl, plus_val, minus_val = outcome
        results.append({"Label": lbl, "plus_test": plus_val, "minus_test": minus_val})
    
    out_df = pd.DataFrame(results, columns=["Label", "plus_test", "minus_test"])
    out_path = f"My_lam_{p}_comps.csv"
    out_df.to_csv(out_path, index=False)

In [1]:
import pandas as pd

primes = prime_range(5,48)
combined_results = []

for p in primes:
    filename = f"My_lam_{p}_comps.csv"
    out_df = pd.read_csv(filename)
    
    count_lam_equal_one = out_df[
        (out_df["plus_test"] == False) & (out_df["minus_test"] == False)
    ].shape[0]

    count_plus_bigger = out_df[
        (out_df["plus_test"] == True)
    ].shape[0]

    count_minus_bigger = out_df[
        (out_df["minus_test"] == True)
    ].shape[0]

    count_both_bigger = out_df[
        (out_df["plus_test"] == True) & (out_df["minus_test"] == True)
    ].shape[0]

    # Calculate the total of the four counts
    total_count = (count_lam_equal_one 
                   + count_plus_bigger 
                   + count_minus_bigger 
                   + count_both_bigger)

    # Store results in a list, placing 'Total' right after 'p'
    combined_results.append([
        p,
        total_count,
        count_lam_equal_one,
        count_plus_bigger,
        count_minus_bigger,
        count_both_bigger
    ])

# Create DataFrame with 'Total' immediately after 'p'
final_df = pd.DataFrame(
    combined_results,
    columns=["p", "Total", "lam+-=1", "lam+>1", "lam_->1", "lam+ and lam- >1"]
)

print(final_df)


     p  Total  lam+-=1  lam+>1  lam_->1  lam+ and lam- >1
0    5   4399     2990     746      568                95
1    7   5051     4261     291      478                21
2   11   6627     5609     580      410                28
3   13   2166     1832     164      161                 9
4   17   3435     3038     197      192                 8
5   19   3689     3523      17      146                 3
6   23   5978     5751      28      198                 1
7   29   2928     2816      18       94                 0
8   31   4119     3995       6      118                 0
9   37    600      589       2        9                 0
10  41   2734     2665      26       43                 0
11  43   1539     1510       0       29                 0
12  47   5212     5129       4       79                 0
